## Usando RAG para Docs de condomínio


In [3]:
# RAG para Documentos de Condomínio - Versão Otimizada

import os
import glob
from dotenv import load_dotenv
import gradio as gr

# Imports LangChain
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import UnstructuredFileLoader
from langchain.prompts import PromptTemplate



In [4]:
# ============================================================================
# 1. CONFIGURAÇÃO INICIAL
# ============================================================================

# Configuração de modelos e diretórios
MODEL = "gpt-4o-mini"
DB_NAME = "vector_db"

# Carregamento de variáveis de ambiente
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')


In [5]:
# ============================================================================
# 2. CARREGAMENTO E PROCESSAMENTO DE DOCUMENTOS
# ============================================================================

def load_documents():
    """Carrega documentos de forma organizada e com metadados apropriados"""
    
    BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../0_base_conhecimento"))
    CACHE_DIR = os.path.join(BASE_DIR, "processed_docs_cache")
    
    print(f"🔍 Verificando diretório: {CACHE_DIR}")
    
    if not os.path.exists(CACHE_DIR):
        print(f"❌ Diretório não encontrado: {CACHE_DIR}")
        return []
    
    print(f"📁 Conteúdo: {os.listdir(CACHE_DIR)}")
    
    documents = []
    text_loader_kwargs = {'encoding': 'utf-8'}
    
    # Busca por subdiretórios (tipos de documento)
    for subfolder in os.listdir(CACHE_DIR):
        subfolder_path = os.path.join(CACHE_DIR, subfolder)
        
        if not os.path.isdir(subfolder_path):
            continue
            
        doc_type = subfolder
        print(f"📄 Processando: {doc_type}")
        
        # Carrega arquivos .txt
        try:
            txt_loader = DirectoryLoader(
                subfolder_path, 
                glob="**/*.txt", 
                loader_cls=TextLoader,
                loader_kwargs=text_loader_kwargs,
                recursive=True
            )
            txt_docs = txt_loader.load()
        except Exception as e:
            print(f"⚠️ Erro ao carregar TXT de {doc_type}: {e}")
            txt_docs = []
        
        # Carrega arquivos .docx
        try:
            docx_loader = DirectoryLoader(
                subfolder_path,
                glob="**/*.docx",
                loader_cls=UnstructuredFileLoader,
                recursive=True
            )
            docx_docs = docx_loader.load()
        except Exception as e:
            print(f"⚠️ Erro ao carregar DOCX de {doc_type}: {e}")
            docx_docs = []
        
        # Processa e adiciona metadados
        for doc in txt_docs + docx_docs:
            # Limpa e padroniza o conteúdo
            content = str(doc.page_content).strip()
            if len(content) < 50:  # Ignora documentos muito pequenos
                continue
                
            # Cria documento com metadados enriquecidos
            new_doc = Document(
                page_content=content,
                metadata={
                    "doc_type": doc_type,
                    "source": doc.metadata.get("source", "unknown"),
                    "filename": os.path.basename(doc.metadata.get("source", "unknown"))
                }
            )
            documents.append(new_doc)
        
        print(f"✅ {doc_type}: {len(txt_docs + docx_docs)} documentos carregados")
    
    print(f"📊 Total de documentos: {len(documents)}")
    return documents

def create_chunks(documents):
    """Cria chunks otimizados para documentos de condomínio"""
    
    if not documents:
        print("❌ Nenhum documento para processar")
        return []
    
    # Splitter otimizado para documentos legais/administrativos
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,  # Menor para melhor precisão
        chunk_overlap=150,  # Overlap maior para manter contexto
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
        length_function=len
    )
    
    chunks = text_splitter.split_documents(documents)
    
    # Adiciona informações extras aos metadados dos chunks
    for i, chunk in enumerate(chunks):
        chunk.metadata["chunk_id"] = i
        chunk.metadata["chunk_size"] = len(chunk.page_content)
    
    print(f"🔗 Total de chunks criados: {len(chunks)}")
    print(f"📋 Tipos de documento encontrados: {set(doc.metadata['doc_type'] for doc in documents)}")
    
    return chunks

In [6]:
# ============================================================================
# 3. CRIAÇÃO DO VECTOR STORE
# ============================================================================

def create_vectorstore(chunks):
    """Cria vectorstore otimizado"""
    
    if not chunks:
        return None
    
    # Embeddings com modelo mais recente
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    # Remove database anterior se existir
    if os.path.exists(DB_NAME):
        import shutil
        shutil.rmtree(DB_NAME)
        print(f"🗑️ Database anterior removido: {DB_NAME}")
    
    # Cria novo vectorstore
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=DB_NAME,
        collection_name="condo_docs"
    )
    
    count = vectorstore._collection.count()
    print(f"💾 Vectorstore criado com {count:,} documentos")
    
    return vectorstore

In [7]:
# ============================================================================
# 4. CONFIGURAÇÃO DO CHAT RAG
# ============================================================================

def setup_rag_chain(vectorstore):
    """Configura a cadeia RAG com prompt customizado"""
    
    # Prompt customizado para documentos de condomínio
    custom_prompt = PromptTemplate(
        template="""Você é um assistente especializado em documentos de condomínio. 
        Use as informações fornecidas para responder de forma precisa e detalhada.

        Contexto dos documentos:
        {context}

        Histórico da conversa:
        {chat_history}

        Pergunta: {question}

        Instruções:
        - Responda baseado APENAS nas informações dos documentos fornecidos
        - Se não encontrar a informação, diga claramente que não está disponível nos documentos
        - Cite o tipo de documento quando relevante (ata, contrato, etc.)
        - Seja preciso com datas, valores e nomes
        - Use formatação clara para listas e informações importantes

        Resposta:""",
        input_variables=["context", "chat_history", "question"]
    )
    
    # LLM
    llm = ChatOpenAI(
        temperature=0.3,  # Baixa temperatura para respostas mais precisas
        model_name=MODEL
    )
    
    # Memória
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True,
        output_key='answer'
    )
    
    # Retriever com configurações otimizadas
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 10,  # Número de chunks a recuperar
            "fetch_k": 20  # Número de chunks a considerar antes de filtrar
        }
    )
    
    # Chain
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        verbose=True
    )
    
    return conversation_chain

In [ ]:
# ============================================================================
# 5. EXECUÇÃO PRINCIPAL
# ============================================================================

# Carregamento e processamento
print("🚀 Iniciando processamento...")

# Passo 1: Carregar documentos
documents = load_documents()
if not documents:
    print("❌ ERRO: Nenhum documento foi carregado!")
    print("Verifique:")
    print("1. Se o diretório '../0_base_conhecimento/processed_docs_cache' existe")
    print("2. Se há arquivos .txt ou .docx nas subpastas")
    print("3. Se os arquivos têm conteúdo suficiente")
    exit()

# Passo 2: Criar chunks
chunks = create_chunks(documents)
if not chunks:
    print("❌ ERRO: Nenhum chunk foi criado!")
    exit()

# Passo 3: Criar vectorstore
vectorstore = create_vectorstore(chunks)
if vectorstore is None:
    print("❌ ERRO: Falha na criação do vectorstore!")
    exit()

# Passo 4: Testar vectorstore
try:
    test_query = vectorstore.similarity_search("assembleia", k=1)
    print(f"✅ Teste do vectorstore: {len(test_query)} resultados encontrados")
except Exception as e:
    print(f"❌ ERRO no teste do vectorstore: {e}")
    exit()

# Passo 5: Configuração do RAG
try:
    conversation_chain = setup_rag_chain(vectorstore)
    print("✅ Sistema RAG configurado com sucesso!")
except Exception as e:
    print(f"❌ ERRO na configuração do RAG: {e}")
    exit()

🚀 Iniciando processamento...
🔍 Verificando diretório: c:\Users\02392179425\Desktop\rag_condominio\0_base_conhecimento\processed_docs_cache
📁 Conteúdo: ['049b078c-b5bf-4d97-bdf7-a2fbd7c66f3b.txt', '065b78d0-c521-4ff9-9932-3825b4b82437.txt', '080a5ffd-2464-40f4-92c0-b4b07e45f617.txt', '2789187e-d466-448c-95ff-f7ff1250a5a4.txt', '280c2644-d328-40d5-bd2c-755a12df2a78.txt', '2ab34b5f-907f-4a18-b0d1-00b152a94df0.txt', '2d264460-0427-482b-ae8d-1c63b20fc53a.txt', '4ef8fafa-78b6-4f4e-8c81-6ca0132fe3b4.txt', '5f64b310-fb6b-412e-b295-96c52a2a1cf4.txt', '5f6bce90-49ed-4d61-858a-fba2f2e1bfe0.txt', '633681a6-5e09-4894-b648-7204c60baea2.txt', '65acae7c-a59f-4e10-8d41-22632869a150.txt', '6917e5df-e366-4198-98b0-1ee290786cb9.txt', '798a79eb-8cbc-4645-9610-474411733ed5.txt', '7cf9f6a5-99d3-4c15-9197-bfb22d2ea64e.txt', '874d4d25-d861-48a4-8060-b2c281d0ff20.txt', '8a71943b-ebd6-45a5-856b-5ead72b58d06.txt', '98e667e9-9dc1-4e00-a6fe-51fb7fa0b1a1.txt', '9c5d36c6-bda9-40df-a782-6f42537facf5.txt', '9c7408f2-66

C:\Users\02392179425\AppData\Local\Temp\ipykernel_27192\3604682758.py:39: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


: 

In [ ]:
# ============================================================================
# 6. TESTE COM PERGUNTAS EXEMPLO
# ============================================================================

def test_questions():
    """Testa o sistema com perguntas exemplo"""
    
    sample_questions = [
        "Quando ocorreu a última assembleia registrada?",
        "Quais são os contratos ativos do condomínio?",
        "Qual o valor atual da taxa condominial?",
        "Quem são os membros do conselho fiscal?",
        "Existe contrato de manutenção de elevadores?"
    ]
    
    print("\n" + "="*60)
    print("🧪 TESTANDO SISTEMA COM PERGUNTAS EXEMPLO")
    print("="*60)
    
    for i, question in enumerate(sample_questions, 1):
        print(f"\n📝 Pergunta {i}: {question}")
        try:
            result = conversation_chain.invoke({"question": question})
            answer = result['answer']
            sources = result.get('source_documents', [])
            
            print(f"💬 Resposta: {answer}")
            
            if sources:
                print(f"📄 Fontes ({len(sources)}): ", end="")
                doc_types = [doc.metadata.get('doc_type', 'unknown') for doc in sources[:3]]
                print(", ".join(set(doc_types)))
            
        except Exception as e:
            print(f"❌ Erro: {e}")
        
        print("-" * 40)

# Executar teste
test_questions()

In [ ]:
# ============================================================================
# 7. INTERFACE GRADIO
# ============================================================================

def chat_function(message, history):
    """Função de chat para Gradio"""
    try:
        result = conversation_chain.invoke({"question": message})
        return result["answer"]
    except Exception as e:
        return f"Erro ao processar pergunta: {str(e)}"

# Criar interface
print("\n🌐 Iniciando interface Gradio...")
interface = gr.ChatInterface(
    fn=chat_function,
    type="messages",
    title="🏢 Assistente de Condomínio RAG",
    description="Faça perguntas sobre os documentos do condomínio (atas, contratos, etc.)",
    examples=[
        "Quando foi a última assembleia?",
        "Quais contratos estão vigentes?",
        "Qual o valor da taxa condominial?",
        "Quem são os síndicos atuais?"
    ]
)

# Lançar interface
interface.launch(
    inbrowser=True,
    share=False,
    server_name="127.0.0.1",
    server_port=7860
)